<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/bonds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper for io::rate like formatting
def format_rate(r):
    return f"{r * 100:.4f}%"

# Helper for ASSERT_CLOSE macro
def assert_close(test_case_instance, name, settlement_date, calculated, expected, tolerance, msg_prefix=""):
    if abs(calculated - expected) > tolerance:
        error_msg = (
            f"{msg_prefix}Failed to reproduce {name} at {settlement_date}\n"
            f"    calculated: {calculated:.9f}\n"
            f"    expected:   {expected:.9f}\n"
            f"    error:      {calculated - expected:.9f}\n"
            f"    tolerance:  {tolerance:.1e}"
        )
        test_case_instance.fail(error_msg)

class CommonVars:
    def __init__(self):
        self.calendar = ql.TARGET()
        # Ensure evaluation date is fixed for reproducibility, matching C++ logic
        # If Date.todaysDate() is used, it must be patched or a fixed date used
        # For this translation, we will use a fixed date if vars.today needs to be dynamic in C++ tests
        # Here, vars.today is calendar.adjust(Date.todaysDate()), so we'll fix it.
        self.today = ql.Date(15, ql.May, 2024) # Example fixed date
        ql.Settings.instance().evaluationDate = self.today
        self.faceAmount = 1000000.0

class BondsTests(unittest.TestCase):

    def checkValue(self, value, expectedValue, tolerance, msg):
        if abs(value - expectedValue) > tolerance:
            self.fail(
                f"{msg}\n"
                f"    calculated: {value:.9f}\n"
                f"    expected:   {expectedValue:.9f}\n"
                f"    tolerance:  {tolerance}\n"
                f"    error:      {value - expectedValue:.9f}"
            )

    def setUp(self):
        """
        This method is called before each test method.
        We use it to reset the evaluation date to a known state.
        """
        self.original_eval_date = ql.Settings.instance().evaluationDate
        # For most tests, a CommonVars instance will set its own today
        # but if a test doesn't, this ensures a baseline.
        # However, many tests in the C++ suite reset the eval date specifically.

    def tearDown(self):
        """
        This method is called after each test method.
        Restore original evaluation date.
        """
        ql.Settings.instance().evaluationDate = self.original_eval_date


    def testYield(self):
        print("Testing consistency of bond price/yield calculation...")
        vars_ = CommonVars()
        original_eval_date = ql.Settings.instance().evaluationDate # Store original
        ql.Settings.instance().evaluationDate = vars_.today # Set to CommonVars today

        tolerance = 1.0e-7
        max_evaluations = 100

        issue_months = [-24, -18, -12, -6, 0, 6, 12, 18, 24]
        lengths = [3, 5, 10, 15, 20]
        settlement_days = 3
        coupons = [0.02, 0.05, 0.08]
        frequencies = [ql.Semiannual, ql.Annual]
        bond_day_count = ql.Thirty360(ql.Thirty360.BondBasis)
        accrual_convention = ql.Unadjusted
        payment_convention = ql.ModifiedFollowing
        redemption = 100.0

        yield_rates = [0.03, 0.04, 0.05, 0.06, 0.07]
        compoundings = [ql.Compounded, ql.Continuous]

        for issue_month_offset in issue_months:
            for length_in_years in lengths:
                for coupon_rate_val in coupons:
                    for freq_val in frequencies:
                        for comp_val in compoundings:
                            dated_date = vars_.calendar.advance(vars_.today, issue_month_offset, ql.Months)
                            issue_date = dated_date
                            maturity_date = vars_.calendar.advance(issue_date, length_in_years, ql.Years)

                            sch = ql.Schedule(dated_date, maturity_date, ql.Period(freq_val),
                                              vars_.calendar, accrual_convention, accrual_convention,
                                              ql.DateGeneration.Backward, False)

                            bond = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch,
                                                    [coupon_rate_val], bond_day_count,
                                                    payment_convention, redemption, issue_date)

                            for market_yield_val in yield_rates:
                                # Test clean price
                                price_obj = ql.BondPrice(
                                    ql.BondFunctions.cleanPrice(bond, market_yield_val, bond_day_count, comp_val, freq_val),
                                    ql.BondPrice.Clean
                                )
                                calculated_yield = ql.BondFunctions.yield_(
                                    bond, price_obj.amount(), bond_day_count, comp_val, freq_val,
                                    ql.Date(), # settlementDate, default means bond.settlementDate()
                                    tolerance, max_evaluations, 0.05, price_obj.type() # Added price type
                                )

                                if abs(market_yield_val - calculated_yield) > tolerance:
                                    price2 = ql.BondFunctions.cleanPrice(bond, calculated_yield, bond_day_count, comp_val, freq_val)
                                    if abs(price_obj.amount() - price2) / price_obj.amount() > tolerance:
                                        self.fail(
                                            f"\nyield recalculation failed (clean price):"
                                            f"\n    issue:        {issue_date}"
                                            f"\n    maturity:     {maturity_date}"
                                            f"\n    coupon:       {format_rate(coupon_rate_val)}"
                                            f"\n    frequency:    {freq_val}"
                                            f"\n    yield:        {format_rate(market_yield_val)}"
                                            f"{' compounded' if comp_val == ql.Compounded else ' continuous'}"
                                            f"\n    clean price:  {price_obj.amount():.7f}"
                                            f"\n    yield':       {format_rate(calculated_yield)}"
                                            f"\n    clean price': {price2:.7f}")

                                # Test dirty price
                                price_obj_dirty = ql.BondPrice(
                                    ql.BondFunctions.dirtyPrice(bond, market_yield_val, bond_day_count, comp_val, freq_val),
                                    ql.BondPrice.Dirty
                                )
                                calculated_yield_dirty = ql.BondFunctions.yield_(
                                    bond, price_obj_dirty.amount(), bond_day_count, comp_val, freq_val,
                                    ql.Date(), tolerance, max_evaluations, 0.05, price_obj_dirty.type() # Added price type
                                )
                                if abs(market_yield_val - calculated_yield_dirty) > tolerance:
                                    price2_dirty = ql.BondFunctions.dirtyPrice(bond, calculated_yield_dirty, bond_day_count, comp_val, freq_val)
                                    if abs(price_obj_dirty.amount() - price2_dirty) / price_obj_dirty.amount() > tolerance:
                                        self.fail(
                                            f"\nyield recalculation failed (dirty price):"
                                            f"\n    issue:        {issue_date}"
                                            f"\n    maturity:     {maturity_date}"
                                            f"\n    coupon:       {format_rate(coupon_rate_val)}"
                                            f"\n    frequency:    {freq_val}"
                                            f"\n    yield:        {format_rate(market_yield_val)}"
                                            f"{' compounded' if comp_val == ql.Compounded else ' continuous'}"
                                            f"\n    dirty price:  {price_obj_dirty.amount():.7f}"
                                            f"\n    yield':       {format_rate(calculated_yield_dirty)}"
                                            f"\n    dirty price': {price2_dirty:.7f}")
        ql.Settings.instance().evaluationDate = original_eval_date # Restore

    def testAtmRate(self):
        print("Testing consistency of bond price/ATM rate calculation...")
        vars_ = CommonVars()
        original_eval_date = ql.Settings.instance().evaluationDate
        ql.Settings.instance().evaluationDate = vars_.today

        tolerance = 1.0e-7
        issue_months = [-24, -18, -12, -6, 0, 6, 12, 18, 24]
        lengths = [3, 5, 10, 15, 20]
        settlement_days = 3
        coupons = [0.02, 0.05, 0.08]
        frequencies = [ql.Semiannual, ql.Annual]
        bond_day_count = ql.Thirty360(ql.Thirty360.BondBasis)
        accrual_convention = ql.Unadjusted
        payment_convention = ql.ModifiedFollowing
        redemption = 100.0

        disc_curve_handle = ql.YieldTermStructureHandle(ql.FlatForward(vars_.today, 0.03, ql.Actual360()))
        bond_engine = ql.DiscountingBondEngine(disc_curve_handle)

        for issue_month_offset in issue_months:
            for length_in_years in lengths:
                for coupon_rate_val in coupons:
                    for freq_val in frequencies:
                        dated_date = vars_.calendar.advance(vars_.today, issue_month_offset, ql.Months)
                        issue_date = dated_date
                        maturity_date = vars_.calendar.advance(issue_date, length_in_years, ql.Years)

                        sch = ql.Schedule(dated_date, maturity_date, ql.Period(freq_val),
                                          vars_.calendar, accrual_convention, accrual_convention,
                                          ql.DateGeneration.Backward, False)

                        bond = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch,
                                                [coupon_rate_val], bond_day_count,
                                                payment_convention, redemption, issue_date)
                        bond.setPricingEngine(bond_engine)

                        price_obj_clean = ql.BondPrice(bond.cleanPrice(), ql.BondPrice.Clean)
                        calculated_atm_rate_clean = ql.BondFunctions.atmRate(
                            bond, disc_curve_handle, bond.settlementDate(), price_obj_clean.amount(), price_obj_clean.type()
                        )
                        if abs(coupon_rate_val - calculated_atm_rate_clean) > tolerance:
                            self.fail(
                                f"\natm rate recalculation failed (clean price):"
                                f"\n today:           {vars_.today}"
                                f"\n settlement date: {bond.settlementDate()}"
                                f"\n issue:           {issue_date}"
                                f"\n maturity:        {maturity_date}"
                                f"\n coupon:          {format_rate(coupon_rate_val)}"
                                f"\n frequency:       {freq_val}"
                                f"\n clean price:     {price_obj_clean.amount()}"
                                f"\n atm rate:        {format_rate(calculated_atm_rate_clean)}")

                        price_obj_dirty = ql.BondPrice(bond.dirtyPrice(), ql.BondPrice.Dirty)
                        calculated_atm_rate_dirty = ql.BondFunctions.atmRate(
                            bond, disc_curve_handle, bond.settlementDate(), price_obj_dirty.amount(), price_obj_dirty.type()
                        )
                        if abs(coupon_rate_val - calculated_atm_rate_dirty) > tolerance:
                            self.fail(
                                f"\natm rate recalculation failed (dirty price):"
                                f"\n today:           {vars_.today}"
                                f"\n settlement date: {bond.settlementDate()}"
                                f"\n issue:           {issue_date}"
                                f"\n maturity:        {maturity_date}"
                                f"\n coupon:          {format_rate(coupon_rate_val)}"
                                f"\n frequency:       {freq_val}"
                                f"\n dirty price:     {price_obj_dirty.amount()}"
                                f"\n atm rate:        {format_rate(calculated_atm_rate_dirty)}")
        ql.Settings.instance().evaluationDate = original_eval_date

    def testZspread(self):
        print("Testing consistency of bond price/z-spread calculation...")
        vars_ = CommonVars()
        original_eval_date = ql.Settings.instance().evaluationDate
        ql.Settings.instance().evaluationDate = vars_.today

        tolerance = 1.0e-7
        max_evaluations = 100
        discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(vars_.today, 0.03, ql.Actual360()))

        issue_months = [-24, -18, -12, -6, 0, 6, 12, 18, 24]
        lengths = [3, 5, 10, 15, 20]
        settlement_days = 3
        coupons = [0.02, 0.05, 0.08]
        frequencies = [ql.Semiannual, ql.Annual]
        bond_day_count = ql.Thirty360(ql.Thirty360.BondBasis)
        accrual_convention = ql.Unadjusted
        payment_convention = ql.ModifiedFollowing
        redemption = 100.0
        spreads = [-0.01, -0.005, 0.0, 0.005, 0.01]
        compoundings = [ql.Compounded, ql.Continuous]

        for issue_month_offset in issue_months:
            for length_in_years in lengths:
                for coupon_rate_val in coupons:
                    for freq_val in frequencies:
                        for comp_val in compoundings:
                            dated_date = vars_.calendar.advance(vars_.today, issue_month_offset, ql.Months)
                            issue_date = dated_date
                            maturity_date = vars_.calendar.advance(issue_date, length_in_years, ql.Years)

                            sch = ql.Schedule(dated_date, maturity_date, ql.Period(freq_val),
                                              vars_.calendar, accrual_convention, accrual_convention,
                                              ql.DateGeneration.Backward, False)
                            bond = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch,
                                                    [coupon_rate_val], bond_day_count,
                                                    payment_convention, redemption, issue_date)
                            for spread_val in spreads:
                                # Clean price
                                price_val_clean = ql.BondFunctions.cleanPrice(
                                    bond, discount_curve, spread_val, bond_day_count, comp_val, freq_val)
                                price_obj_clean = ql.BondPrice(price_val_clean, ql.BondPrice.Clean)

                                calculated_spread_clean = ql.BondFunctions.zSpread(
                                    bond, price_obj_clean.amount(), discount_curve, bond_day_count, comp_val, freq_val,
                                    ql.Date(), tolerance, max_evaluations, 0.0, price_obj_clean.type() # guess 0.0, price type
                                )
                                if abs(spread_val - calculated_spread_clean) > tolerance:
                                    price2_clean = ql.BondFunctions.cleanPrice(
                                        bond, discount_curve, calculated_spread_clean, bond_day_count, comp_val, freq_val)
                                    if abs(price_obj_clean.amount() - price2_clean) / price_obj_clean.amount() > tolerance:
                                        self.fail(
                                            f"\nZ-spread recalculation failed (clean price):"
                                            f"\n    issue:     {issue_date}"
                                            f"\n    maturity:  {maturity_date}"
                                            f"\n    coupon:    {format_rate(coupon_rate_val)}"
                                            f"\n    frequency: {freq_val}"
                                            f"\n    Z-spread:  {format_rate(spread_val)}"
                                            f"{' compounded' if comp_val == ql.Compounded else ' continuous'}"
                                            f"\n    clean price:  {price_obj_clean.amount():.7f}"
                                            f"\n    Z-spread': {format_rate(calculated_spread_clean)}"
                                            f"\n    clean price': {price2_clean:.7f}")

                                # Dirty price
                                price_val_dirty = ql.BondFunctions.dirtyPrice(
                                    bond, discount_curve, spread_val, bond_day_count, comp_val, freq_val)
                                price_obj_dirty = ql.BondPrice(price_val_dirty, ql.BondPrice.Dirty)

                                calculated_spread_dirty = ql.BondFunctions.zSpread(
                                    bond, price_obj_dirty.amount(), discount_curve, bond_day_count, comp_val, freq_val,
                                    ql.Date(), tolerance, max_evaluations, 0.0, price_obj_dirty.type() # guess 0.0, price type
                                )
                                if abs(spread_val - calculated_spread_dirty) > tolerance:
                                    price2_dirty = ql.BondFunctions.dirtyPrice(
                                        bond, discount_curve, calculated_spread_dirty, bond_day_count, comp_val, freq_val)
                                    if abs(price_obj_dirty.amount() - price2_dirty) / price_obj_dirty.amount() > tolerance:
                                        self.fail(
                                            f"\nZ-spread recalculation failed (dirty price):"
                                            f"\n    issue:        {issue_date}"
                                            f"\n    maturity:     {maturity_date}"
                                            f"\n    coupon:       {format_rate(coupon_rate_val)}"
                                            f"\n    frequency:    {freq_val}"
                                            f"\n    Z-spread:     {format_rate(spread_val)}"
                                            f"{' compounded' if comp_val == ql.Compounded else ' continuous'}"
                                            f"\n    dirty price:  {price_obj_dirty.amount():.7f}"
                                            f"\n    Z-spread':    {format_rate(calculated_spread_dirty)}"
                                            f"\n    dirty price': {price2_dirty:.7f}")
        ql.Settings.instance().evaluationDate = original_eval_date


    def testTheoretical(self):
        print("Testing theoretical bond price/yield calculation...")
        vars_ = CommonVars()
        original_eval_date = ql.Settings.instance().evaluationDate
        ql.Settings.instance().evaluationDate = vars_.today

        tolerance = 1.0e-7
        max_evaluations = 100

        lengths = [3, 5, 10, 15, 20]
        settlement_days = 3
        coupons = [0.02, 0.05, 0.08]
        frequencies = [ql.Semiannual, ql.Annual]
        bond_day_count = ql.Actual360()
        accrual_convention = ql.Unadjusted
        payment_convention = ql.ModifiedFollowing
        redemption = 100.0
        yield_rates = [0.03, 0.04, 0.05, 0.06, 0.07]

        for length_in_years in lengths:
            for coupon_rate_val in coupons:
                for freq_val in frequencies:
                    dated_date = vars_.today
                    issue_date = dated_date
                    maturity_date = vars_.calendar.advance(issue_date, length_in_years, ql.Years)

                    rate_quote = ql.SimpleQuote(0.0)
                    discount_curve = ql.YieldTermStructureHandle(
                        ql.FlatForward(vars_.today, rate_quote, bond_day_count))

                    sch = ql.Schedule(dated_date, maturity_date, ql.Period(freq_val),
                                      vars_.calendar, accrual_convention, accrual_convention,
                                      ql.DateGeneration.Backward, False)
                    bond = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch,
                                            [coupon_rate_val], bond_day_count, payment_convention,
                                            redemption, issue_date)
                    bond_engine = ql.DiscountingBondEngine(discount_curve)
                    bond.setPricingEngine(bond_engine)

                    for market_yield_val in yield_rates:
                        rate_quote.setValue(market_yield_val)

                        # Test yield vs clean price
                        price_val = ql.BondFunctions.cleanPrice(
                            bond, market_yield_val, bond_day_count, ql.Continuous, freq_val)
                        calculated_price = bond.cleanPrice()
                        if abs(price_val - calculated_price) > tolerance:
                            self.fail(
                                f"price calculation failed (clean price vs yield):"
                                f"\n    issue:       {issue_date}"
                                f"\n    maturity:    {maturity_date}"
                                f"\n    coupon:      {format_rate(coupon_rate_val)}"
                                f"\n    frequency:   {freq_val}"
                                f"\n    yield:       {format_rate(market_yield_val)}"
                                f"\n    expected:    {price_val:.7f}"
                                f"\n    calculated': {calculated_price:.7f}"
                                f"\n    error':      {price_val - calculated_price:.7f}")

                        calculated_yield = ql.BondFunctions.yield_(
                            bond, calculated_price, bond_day_count, ql.Continuous, freq_val,
                            bond.settlementDate(), tolerance, max_evaluations, 0.0, ql.BondPrice.Clean # Price type
                        )
                        if abs(market_yield_val - calculated_yield) > tolerance:
                            self.fail(
                                f"yield calculation failed (clean price vs yield):"
                                f"\n    issue:     {issue_date}"
                                f"\n    maturity:  {maturity_date}"
                                f"\n    coupon:    {format_rate(coupon_rate_val)}"
                                f"\n    frequency: {freq_val}"
                                f"\n    yield:     {format_rate(market_yield_val)}"
                                f"\n    clean price: {price_val:.7f}"
                                f"\n    yield':    {format_rate(calculated_yield)}")

                        # Test yield vs dirty price
                        price_val_dirty = ql.BondFunctions.dirtyPrice(
                            bond, market_yield_val, bond_day_count, ql.Continuous, freq_val)
                        calculated_price_dirty = bond.dirtyPrice()
                        if abs(price_val_dirty - calculated_price_dirty) > tolerance:
                            self.fail(
                                f"price calculation failed (dirty price vs yield):"
                                f"\n    issue:       {issue_date}"
                                f"\n    maturity:    {maturity_date}"
                                f"\n    coupon:      {format_rate(coupon_rate_val)}"
                                f"\n    frequency:   {freq_val}"
                                f"\n    yield:       {format_rate(market_yield_val)}"
                                f"\n    expected:    {price_val_dirty:.7f}"
                                f"\n    calculated': {calculated_price_dirty:.7f}"
                                f"\n    error':      {price_val_dirty - calculated_price_dirty:.7f}")

                        calculated_yield_dirty = ql.BondFunctions.yield_(
                            bond, calculated_price_dirty, bond_day_count, ql.Continuous, freq_val,
                            bond.settlementDate(), tolerance, max_evaluations, 0.05, ql.BondPrice.Dirty # Price type
                        )
                        if abs(market_yield_val - calculated_yield_dirty) > tolerance:
                            self.fail(
                                f"yield calculation failed (dirty price vs yield):"
                                f"\n    issue:       {issue_date}"
                                f"\n    maturity:    {maturity_date}"
                                f"\n    coupon:      {format_rate(coupon_rate_val)}"
                                f"\n    frequency:   {freq_val}"
                                f"\n    yield:       {format_rate(market_yield_val)}"
                                f"\n    dirty price: {price_val_dirty:.7f}"
                                f"\n    yield':      {format_rate(calculated_yield_dirty)}")
        ql.Settings.instance().evaluationDate = original_eval_date

    def testCached(self):
        print("Testing bond price/yield calculation against cached values...")
        vars_ = CommonVars()
        original_eval_date = ql.Settings.instance().evaluationDate

        today = ql.Date(22, ql.November, 2004)
        ql.Settings.instance().evaluationDate = today

        bond_calendar = ql.NullCalendar()
        settlement_days = 1
        discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.03, ql.Actual360()))
        freq = ql.Semiannual
        tolerance = 1.0e-6

        # Bond 1
        sch1_date = ql.Date(31, ql.October, 2004)
        sch1_mat = ql.Date(31, ql.October, 2006)
        sch1 = ql.Schedule(sch1_date, sch1_mat, ql.Period(freq), bond_calendar,
                           ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, True)
        bond_day_count1 = ql.ActualActual(ql.ActualActual.ISMA, sch1)
        bond_day_count1_no_schedule = ql.ActualActual(ql.ActualActual.ISMA)

        bond1 = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch1, [0.025],
                                 bond_day_count1, ql.ModifiedFollowing, 100.0, ql.Date(1, ql.November, 2004))
        bond1_no_schedule = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch1, [0.025],
                                            bond_day_count1_no_schedule, ql.ModifiedFollowing,
                                            100.0, ql.Date(1, ql.November, 2004))

        bond_engine = ql.DiscountingBondEngine(discount_curve)
        bond1.setPricingEngine(bond_engine)
        bond1_no_schedule.setPricingEngine(bond_engine)

        market_price1 = 99.203125
        market_yield1 = 0.02925

        # Bond 2
        sch2_date = ql.Date(15, ql.November, 2004)
        sch2_mat = ql.Date(15, ql.November, 2009)
        sch2 = ql.Schedule(sch2_date, sch2_mat, ql.Period(freq), bond_calendar,
                           ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, False)
        bond_day_count2 = ql.ActualActual(ql.ActualActual.ISMA, sch2)
        bond_day_count2_no_schedule = ql.ActualActual(ql.ActualActual.ISMA)

        bond2 = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch2, [0.035],
                                 bond_day_count2, ql.ModifiedFollowing, 100.0, ql.Date(15, ql.November, 2004))
        bond2_no_schedule = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch2, [0.035],
                                            bond_day_count2_no_schedule, ql.ModifiedFollowing,
                                            100.0, ql.Date(15, ql.November, 2004))
        bond2.setPricingEngine(bond_engine)
        bond2_no_schedule.setPricingEngine(bond_engine)

        market_price2 = 99.6875
        market_yield2 = 0.03569

        # Cached values
        cached_price1a = 99.204505; cached_price2a = 99.687192
        cached_price1b = 98.943393; cached_price2b = 101.986794
        cached_yield1a = 0.029257;  cached_yield2a = 0.035689
        cached_yield1b = 0.029045;  cached_yield2b = 0.035375
        cached_yield1c = 0.030423;  cached_yield2c = 0.030432

        # Checks for Bond 1
        self.checkValue(ql.BondFunctions.cleanPrice(bond1, market_yield1, bond_day_count1, ql.Compounded, freq),
                        cached_price1a, tolerance, "failed to reproduce cached price with schedule for bond 1:")
        self.checkValue(ql.BondFunctions.cleanPrice(bond1_no_schedule, market_yield1, bond_day_count1_no_schedule, ql.Compounded, freq),
                        cached_price1a, tolerance, "failed to reproduce cached price with no schedule for bond 1:")
        self.checkValue(bond1.cleanPrice(), cached_price1b, tolerance,
                        "failed to reproduce cached clean price with schedule for bond 1:")
        self.checkValue(bond1_no_schedule.cleanPrice(), cached_price1b, tolerance,
                        "failed to reproduce cached clean price with no schedule for bond 1:")

        price_obj1 = ql.BondPrice(market_price1, ql.BondPrice.Clean)
        self.checkValue(ql.BondFunctions.yield_(bond1, price_obj1.amount(), bond_day_count1, ql.Compounded, freq, ql.Date(), 1e-7, 100, 0.05, price_obj1.type()),
                        cached_yield1a, tolerance, "failed to reproduce cached compounded yield with schedule for bond 1:")
        self.checkValue(ql.BondFunctions.yield_(bond1_no_schedule, price_obj1.amount(), bond_day_count1_no_schedule, ql.Compounded, freq, ql.Date(), 1e-7, 100, 0.05, price_obj1.type()),
                        cached_yield1a, tolerance, "failed to reproduce cached compounded yield with no schedule for bond 1:")
        self.checkValue(ql.BondFunctions.yield_(bond1, price_obj1.amount(), bond_day_count1, ql.Continuous, freq, ql.Date(), 1e-7, 100, 0.05, price_obj1.type()),
                        cached_yield1b, tolerance, "failed to reproduce cached continuous yield with schedule for bond 1:")
        self.checkValue(ql.BondFunctions.yield_(bond1_no_schedule, price_obj1.amount(), bond_day_count1_no_schedule, ql.Continuous, freq, ql.Date(), 1e-7, 100, 0.05, price_obj1.type()),
                        cached_yield1b, tolerance, "failed to reproduce cached continuous yield with no schedule for bond 1:")

        price_obj1_engine = ql.BondPrice(bond1.cleanPrice(), ql.BondPrice.Clean)
        self.checkValue(ql.BondFunctions.yield_(bond1, price_obj1_engine.amount(), bond_day_count1, ql.Continuous, freq, bond1.settlementDate(), 1e-7, 100, 0.05, price_obj1_engine.type()),
                        cached_yield1c, tolerance, "failed to reproduce cached continuous yield with schedule for bond 1 (engine price):")
        price_obj1_no_sched_engine = ql.BondPrice(bond1_no_schedule.cleanPrice(), ql.BondPrice.Clean)
        self.checkValue(ql.BondFunctions.yield_(bond1_no_schedule, price_obj1_no_sched_engine.amount(), bond_day_count1_no_schedule, ql.Continuous, freq, bond1.settlementDate(), 1e-7, 100, 0.05, price_obj1_no_sched_engine.type()),
                        cached_yield1c, tolerance, "failed to reproduce cached continuous yield with no schedule for bond 1 (engine price):")


        # Checks for Bond 2
        self.checkValue(ql.BondFunctions.cleanPrice(bond2, market_yield2, bond_day_count2, ql.Compounded, freq),
                        cached_price2a, tolerance, "failed to reproduce cached price with schedule for bond 2")
        self.checkValue(ql.BondFunctions.cleanPrice(bond2_no_schedule, market_yield2, bond_day_count2_no_schedule, ql.Compounded, freq),
                        cached_price2a, tolerance, "failed to reproduce cached price with no schedule for bond 2:")
        self.checkValue(bond2.cleanPrice(), cached_price2b, tolerance,
                        "failed to reproduce cached clean price with schedule for bond 2:")
        self.checkValue(bond2_no_schedule.cleanPrice(), cached_price2b, tolerance,
                        "failed to reproduce cached clean price with no schedule for bond 2:")

        price_obj2 = ql.BondPrice(market_price2, ql.BondPrice.Clean)
        self.checkValue(ql.BondFunctions.yield_(bond2, price_obj2.amount(), bond_day_count2, ql.Compounded, freq, ql.Date(), 1e-7, 100, 0.05, price_obj2.type()),
                        cached_yield2a, tolerance, "failed to reproduce cached compounded yield with schedule for bond 2:")
        self.checkValue(ql.BondFunctions.yield_(bond2_no_schedule, price_obj2.amount(), bond_day_count2_no_schedule, ql.Compounded, freq, ql.Date(), 1e-7, 100, 0.05, price_obj2.type()),
                        cached_yield2a, tolerance, "failed to reproduce cached compounded yield with no schedule for bond 2:")
        self.checkValue(ql.BondFunctions.yield_(bond2, price_obj2.amount(), bond_day_count2, ql.Continuous, freq, ql.Date(), 1e-7, 100, 0.05, price_obj2.type()),
                        cached_yield2b, tolerance, "failed to reproduce cached continuous yield with schedule for bond 2:")
        self.checkValue(ql.BondFunctions.yield_(bond2_no_schedule, price_obj2.amount(), bond_day_count2_no_schedule, ql.Continuous, freq, ql.Date(), 1e-7, 100, 0.05, price_obj2.type()),
                        cached_yield2b, tolerance, "failed to reproduce cached continuous yield with no schedule for bond 2:")

        price_obj2_engine = ql.BondPrice(bond2.cleanPrice(), ql.BondPrice.Clean)
        self.checkValue(ql.BondFunctions.yield_(bond2, price_obj2_engine.amount(), bond_day_count2, ql.Continuous, freq, bond2.settlementDate(), 1e-7, 100, 0.05, price_obj2_engine.type()),
                        cached_yield2c, tolerance, "failed to reproduce cached continuous yield for bond 2 with schedule (engine price):")
        price_obj2_no_sched_engine = ql.BondPrice(bond2_no_schedule.cleanPrice(), ql.BondPrice.Clean)
        self.checkValue(ql.BondFunctions.yield_(bond2_no_schedule, price_obj2_no_sched_engine.amount(), bond_day_count2_no_schedule, ql.Continuous, freq, bond2_no_schedule.settlementDate(), 1e-7, 100, 0.05, price_obj2_no_sched_engine.type()),
                        cached_yield2c, tolerance, "failed to reproduce cached continuous yield for bond 2 with no schedule (engine price):")

        # With explicit settlement date
        sch3_date = ql.Date(30, ql.November, 2004)
        sch3_mat = ql.Date(30, ql.November, 2006)
        sch3 = ql.Schedule(sch3_date, sch3_mat, ql.Period(freq),
                           ql.UnitedStates(ql.UnitedStates.GovernmentBond),
                           ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, False)
        bond_day_count3 = ql.ActualActual(ql.ActualActual.ISMA, sch3)
        bond_day_count3_no_schedule = ql.ActualActual(ql.ActualActual.ISMA)

        bond3 = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch3, [0.02875],
                                 bond_day_count3, ql.ModifiedFollowing, 100.0, ql.Date(30, ql.November, 2004))
        bond3_no_schedule = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch3, [0.02875],
                                            bond_day_count3_no_schedule, ql.ModifiedFollowing,
                                            100.0, ql.Date(30, ql.November, 2004))
        bond3.setPricingEngine(bond_engine)
        bond3_no_schedule.setPricingEngine(bond_engine)

        market_yield3 = 0.02997
        settlement_date_explicit = ql.Date(30, ql.November, 2004)
        cached_price3 = 99.764759

        self.checkValue(ql.BondFunctions.cleanPrice(bond3, market_yield3, bond_day_count3, ql.Compounded, freq, settlement_date_explicit),
                        cached_price3, tolerance, "Failed to reproduce cached price for bond 3 with schedule")
        self.checkValue(ql.BondFunctions.cleanPrice(bond3_no_schedule, market_yield3, bond_day_count3_no_schedule, ql.Compounded, freq, settlement_date_explicit),
                        cached_price3, tolerance, "Failed to reproduce cached price for bond 3 with no schedule")

        # This should give the same result since the issue date is the earliest possible settlement date
        ql.Settings.instance().evaluationDate = ql.Date(22, ql.November, 2004)
        self.checkValue(ql.BondFunctions.cleanPrice(bond3, market_yield3, bond_day_count3, ql.Compounded, freq), # Implicit settlement
                        cached_price3, tolerance, "Failed to reproduce the cached price for bond 3 with schedule and the earliest possible settlement date")
        self.checkValue(ql.BondFunctions.cleanPrice(bond3_no_schedule, market_yield3, bond_day_count3_no_schedule, ql.Compounded, freq), # Implicit settlement
                        cached_price3, tolerance, "Failed to reproduce the cached price for bond 3 with no schedule and the earliest possible settlement date")

        ql.Settings.instance().evaluationDate = original_eval_date


    def testCachedZero(self):
        print("Testing zero-coupon bond prices against cached values...")
        vars_ = CommonVars()
        original_eval_date = ql.Settings.instance().evaluationDate

        today = ql.Date(22, ql.November, 2004)
        ql.Settings.instance().evaluationDate = today
        settlement_days = 1
        discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.03, ql.Actual360()))
        tolerance = 1.0e-6
        bond_engine = ql.DiscountingBondEngine(discount_curve)

        # Bond 1
        bond1 = ql.ZeroCouponBond(settlement_days, ql.UnitedStates(ql.UnitedStates.GovernmentBond),
                                  vars_.faceAmount, ql.Date(30, ql.November, 2008),
                                  ql.ModifiedFollowing, 100.0, ql.Date(30, ql.November, 2004))
        bond1.setPricingEngine(bond_engine)
        cached_price1 = 88.551726
        self.checkValue(bond1.cleanPrice(), cached_price1, tolerance, "failed to reproduce cached price for zero bond 1:")

        # Bond 2
        bond2 = ql.ZeroCouponBond(settlement_days, ql.UnitedStates(ql.UnitedStates.GovernmentBond),
                                  vars_.faceAmount, ql.Date(30, ql.November, 2007),
                                  ql.ModifiedFollowing, 100.0, ql.Date(30, ql.November, 2004))
        bond2.setPricingEngine(bond_engine)
        cached_price2 = 91.278949
        self.checkValue(bond2.cleanPrice(), cached_price2, tolerance, "failed to reproduce cached price for zero bond 2:")

        # Bond 3
        bond3 = ql.ZeroCouponBond(settlement_days, ql.UnitedStates(ql.UnitedStates.GovernmentBond),
                                  vars_.faceAmount, ql.Date(30, ql.November, 2006),
                                  ql.ModifiedFollowing, 100.0, ql.Date(30, ql.November, 2004))
        bond3.setPricingEngine(bond_engine)
        cached_price3 = 94.098006
        self.checkValue(bond3.cleanPrice(), cached_price3, tolerance, "failed to reproduce cached price for zero bond 3:")

        ql.Settings.instance().evaluationDate = original_eval_date

    def testCachedFixed(self):
        print("Testing fixed-coupon bond prices against cached values...")
        vars_ = CommonVars()
        original_eval_date = ql.Settings.instance().evaluationDate

        today = ql.Date(22, ql.November, 2004)
        ql.Settings.instance().evaluationDate = today
        settlement_days = 1
        discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.03, ql.Actual360()))
        tolerance = 1.0e-6
        bond_engine = ql.DiscountingBondEngine(discount_curve)

        # Plain
        sch = ql.Schedule(ql.Date(30, ql.November, 2004), ql.Date(30, ql.November, 2008),
                          ql.Period(ql.Semiannual), ql.UnitedStates(ql.UnitedStates.GovernmentBond),
                          ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, False)
        bond1 = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch, [0.02875],
                                 ql.ActualActual(ql.ActualActual.ISMA), ql.ModifiedFollowing,
                                 100.0, ql.Date(30, ql.November, 2004))
        bond1.setPricingEngine(bond_engine)
        cached_price1 = 99.298100
        self.checkValue(bond1.cleanPrice(), cached_price1, tolerance, "failed to reproduce cached price for fixed bond 1:")

        # Varying coupons
        coupon_rates = [0.02875, 0.03, 0.03125, 0.0325]
        bond2 = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch, coupon_rates,
                                 ql.ActualActual(ql.ActualActual.ISMA), ql.ModifiedFollowing,
                                 100.0, ql.Date(30, ql.November, 2004))
        bond2.setPricingEngine(bond_engine)
        cached_price2 = 100.334149
        self.checkValue(bond2.cleanPrice(), cached_price2, tolerance, "failed to reproduce cached price for fixed bond 2 (varying coupons):")

        # Stub date
        sch3 = ql.Schedule(ql.Date(30, ql.November, 2004), ql.Date(30, ql.March, 2009),
                           ql.Period(ql.Semiannual), ql.UnitedStates(ql.UnitedStates.GovernmentBond),
                           ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, False,
                           ql.Date(), ql.Date(30, ql.November, 2008)) # NextToLastDate, EndDate
        bond3 = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch3, coupon_rates,
                                 ql.ActualActual(ql.ActualActual.ISMA), ql.ModifiedFollowing,
                                 100.0, ql.Date(30, ql.November, 2004))
        bond3.setPricingEngine(bond_engine)
        cached_price3 = 100.382794
        self.checkValue(bond3.cleanPrice(), cached_price3, tolerance, "failed to reproduce cached price for fixed bond 3 (stub date):")

        ql.Settings.instance().evaluationDate = original_eval_date


    def testCachedFloating(self):
        print("Testing floating-rate bond prices against cached values...")
        vars_ = CommonVars()
        original_eval_date = ql.Settings.instance().evaluationDate
        using_at_par_coupons = ql.IborCoupon.Settings.instance().usingAtParCoupons()

        today = ql.Date(22, ql.November, 2004)
        ql.Settings.instance().evaluationDate = today
        settlement_days = 1

        risk_free_rate = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.025, ql.Actual360()))
        discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.03, ql.Actual360()))

        index = ql.USDLibor(ql.Period(6, ql.Months), risk_free_rate)
        fixing_days = 1
        tolerance = 1.0e-6

        # In Python, optionlet vol structure handle can be default-constructed if not used.
        # However, BlackIborCouponPricer requires a valid handle.
        # Create a dummy flat vol structure for the pricer if no specific vol is implied by the test.
        dummy_vol_ts = ql.OptionletVolatilityStructureHandle(
            ql.ConstantOptionletVolatility(settlement_days, index.fixingCalendar(), ql.Following, 0.0, index.dayCounter())
        )
        pricer = ql.BlackIborCouponPricer(dummy_vol_ts)


        # Plain
        sch = ql.Schedule(ql.Date(30, ql.November, 2004), ql.Date(30, ql.November, 2008),
                          ql.Period(ql.Semiannual), ql.UnitedStates(ql.UnitedStates.GovernmentBond),
                          ql.ModifiedFollowing, ql.ModifiedFollowing, ql.DateGeneration.Backward, False)

        bond1 = ql.FloatingRateBond(settlement_days, vars_.faceAmount, sch, index,
                                    ql.ActualActual(ql.ActualActual.ISMA), ql.ModifiedFollowing, fixing_days,
                                    [], [], [], [], False, 100.0, ql.Date(30, ql.November, 2004))
        bond_engine1 = ql.DiscountingBondEngine(risk_free_rate) # Discount with riskFreeRate
        bond1.setPricingEngine(bond_engine1)
        ql.setCouponPricer(bond1.cashflows(), pricer)

        cached_price1 = 99.874646 if using_at_par_coupons else 99.874645
        self.checkValue(bond1.cleanPrice(), cached_price1, tolerance, "failed to reproduce cached price for floating bond 1 (plain):")

        # Different risk-free and discount curve
        bond2 = ql.FloatingRateBond(settlement_days, vars_.faceAmount, sch, index,
                                    ql.ActualActual(ql.ActualActual.ISMA), ql.ModifiedFollowing, fixing_days,
                                    [], [], [], [], False, 100.0, ql.Date(30, ql.November, 2004))
        bond_engine2 = ql.DiscountingBondEngine(discount_curve) # Discount with discountCurve
        bond2.setPricingEngine(bond_engine2)
        ql.setCouponPricer(bond2.cashflows(), pricer)

        cached_price2 = 97.955904
        self.checkValue(bond2.cleanPrice(), cached_price2, tolerance, "failed to reproduce cached price for floating bond 2 (different curves):")

        # Varying spread
        spreads = [0.001, 0.0012, 0.0014, 0.0016]
        bond3 = ql.FloatingRateBond(settlement_days, vars_.faceAmount, sch, index,
                                    ql.ActualActual(ql.ActualActual.ISMA), ql.ModifiedFollowing, fixing_days,
                                    [], spreads, [], [], False, 100.0, ql.Date(30, ql.November, 2004))
        bond3.setPricingEngine(bond_engine2) # Discount with discountCurve
        ql.setCouponPricer(bond3.cashflows(), pricer)

        cached_price3 = 98.495459 if using_at_par_coupons else 98.495458
        self.checkValue(bond3.cleanPrice(), cached_price3, tolerance, "failed to reproduce cached price for floating bond 3 (varying spread):")

        # Bond with past fixing
        sch2 = ql.Schedule(ql.Date(26, ql.November, 2003), ql.Date(26, ql.November, 2007),
                           ql.Period(ql.Semiannual), ql.UnitedStates(ql.UnitedStates.GovernmentBond),
                           ql.ModifiedFollowing, ql.ModifiedFollowing, ql.DateGeneration.Backward, False)
        # Note: C++ passes issueDate and then an explicit settlementDateOffset for ex-coupon calc
        # Python's FloatingRateBond doesn't have direct settlementDateOffset. Ex-coupon logic is handled by calendar & period.
        # The Period(6*Days) in C++ is likely an ex-coupon period.
        # Python FixedRateBond has exCouponPeriod, but FloatingRateBond does not.
        # This might be a difference or handled implicitly by the pricing of the first coupon.
        # For simplicity, we'll omit ex-coupon specific adjustments for FloatingRateBond here unless it becomes an issue.
        bond4 = ql.FloatingRateBond(settlement_days, vars_.faceAmount, sch2, index,
                                    ql.ActualActual(ql.ActualActual.ISMA), ql.ModifiedFollowing, fixing_days,
                                    [], spreads, [], [], False, 100.0, ql.Date(29, ql.October, 2004))

        index.addFixing(ql.Date(25, ql.May, 2004), 0.0402)
        bond4.setPricingEngine(bond_engine2)
        ql.setCouponPricer(bond4.cashflows(), pricer)

        cached_price4 = 98.892055 if using_at_par_coupons else 98.892346
        self.checkValue(bond4.cleanPrice(), cached_price4, tolerance, "failed to reproduce cached price for floating bond 4 (past fixing):")

        ql.Settings.instance().evaluationDate = original_eval_date


    def testBrazilianCached(self):
        print("Testing Brazilian public bond prices against Andima cached values...")
        vars_ = CommonVars()
        original_eval_date = ql.Settings.instance().evaluationDate

        settlement_days = 1
        face_amount = 1000.0
        today = ql.Date(6, ql.June, 2007)
        issue_date_bond = ql.Date(1, ql.January, 2007)
        tolerance = 1.0e-4 # High tolerance as per C++
        ql.Settings.instance().evaluationDate = today

        maturity_dates_data = [
            ql.Date(1, ql.January, 2008), ql.Date(1, ql.January, 2010), ql.Date(1, ql.July, 2010),
            ql.Date(1, ql.January, 2012), ql.Date(1, ql.January, 2014), ql.Date(1, ql.January, 2017)
        ]
        yields_data = [0.114614, 0.105726, 0.105328, 0.104283, 0.103218, 0.102948]
        prices_data = [1034.63031372, 1030.09919487, 1029.98307160, 1028.13585068, 1028.33383817, 1026.19716497]

        # coupon_rates_obj = [ql.InterestRate(0.1, ql.Thirty360(ql.Thirty360.BondBasis), ql.Compounded, ql.Annual)]
        # In Python, FixedRateLeg takes a list of rates (float), not InterestRate objects directly for withCouponRates.
        # The InterestRate object is used for the yield itself.
        coupon_rates_float = [0.1]


        for i in range(len(maturity_dates_data)):
            market_yield_obj = ql.InterestRate(yields_data[i], ql.Business252(ql.Brazil()),
                                           ql.Compounded, ql.Annual)

            schedule = ql.Schedule(issue_date_bond, maturity_dates_data[i], ql.Period(ql.Semiannual),
                                   ql.Brazil(ql.Brazil.Settlement), ql.Unadjusted, ql.Unadjusted,
                                   ql.DateGeneration.Backward, False)

            # In C++, a Leg is created then passed to Bond. Python's Bond constructors typically take Schedule directly.
            # For a standard FixedRateBond, we don't build a Leg separately usually.
            # The C++ test uses a generic Bond object with a custom leg.
            # We can simulate this by creating FixedRateBond and extracting its leg,
            # or using BondFunctions which operate on a Bond object.
            # Let's use FixedRateBond as it's common.
            # The C++ code does `Bond bond(settlementDays, schedule.calendar(), issueDate, coupons);`
            # This implies a Bond object constructed from a Leg. We will use FixedRateBond for simplicity and check BondFunctions.

            # The DayCounter for coupons in the C++ test comes from couponRates[0].dayCounter(), which is Thirty360.
            # The DayCounter for yield is Business252.
            # FixedRateBond needs a DayCounter for its coupons.
            temp_bond = ql.FixedRateBond(settlement_days, face_amount, schedule, coupon_rates_float,
                                         ql.Thirty360(ql.Thirty360.BondBasis), # DayCounter for coupons
                                         ql.Following, # Payment convention
                                         100.0, # Redemption per 100 face
                                         issue_date_bond)


            # Price is per 100 face in BondFunctions, then scaled by faceAmount/100
            # Accrued is also per 100 face
            # Clean price from BondFunctions:
            clean_price_per_100 = ql.BondFunctions.cleanPrice(
                temp_bond, market_yield_obj.rate(), market_yield_obj.dayCounter(),
                market_yield_obj.compounding(), market_yield_obj.frequency(),
                today # settlement date
            )

            # Accrued amount for settlement on 'today' (as per C++ test logic)
            accrued_amount_per_100 = temp_bond.accruedAmount(today)

            # Total price (dirty) per 100 face
            dirty_price_per_100 = clean_price_per_100 + accrued_amount_per_100

            # Scale to actual face amount
            calculated_price = dirty_price_per_100 * (face_amount / 100.0)

            expected_price = prices_data[i]

            self.checkValue(calculated_price, expected_price, tolerance,
                            f"failed to reproduce Andima cached price for bond index {i}:")

        ql.Settings.instance().evaluationDate = original_eval_date

    def testExCouponGilt(self):
        print("Testing ex-coupon UK Gilt price against market values...")
        original_eval_date = ql.Settings.instance().evaluationDate

        class TestCaseData:
            def __init__(self, settlementDate, testPrice, accruedAmount, NPV, yield_rate, duration, convexity):
                self.settlementDate = settlementDate; self.testPrice = testPrice
                self.accruedAmount = accruedAmount; self.NPV = NPV
                self.yield_rate = yield_rate; self.duration = duration; self.convexity = convexity

        cases = [
            TestCaseData(ql.Date(29,ql.May,2013), 103.0, 3.8021978, 106.8021978, 0.0749518, 5.6760445, 42.1531486),
            TestCaseData(ql.Date(30,ql.May,2013), 103.0, -0.1758242, 102.8241758, 0.0749618, 5.8928163, 43.7562186),
            TestCaseData(ql.Date(31,ql.May,2013), 103.0, -0.1538462, 102.8461538, 0.0749599, 5.8901860, 43.7239438)
        ]

        calendar = ql.UnitedKingdom()
        settlement_days = 3 # Not directly used when settlementDate is explicit in BondFunctions

        issue_date = ql.Date(29, ql.February, 1996)
        start_date = ql.Date(29, ql.February, 1996) # Accrual start
        first_coupon_date = ql.Date(7, ql.June, 1996)
        maturity_date = ql.Date(7, ql.June, 2021)
        coupon_rate = 0.08
        tenor = ql.Period(6, ql.Months)
        # Ex-coupon period settings from C++:
        # exCouponPeriod = 6*Days, exCouponCalendar = calendar (UK), exCouponConvention = Unadjusted, exCouponEndOfMonth = false
        ex_coupon_period = ql.Period(6, ql.Days)
        ex_coupon_calendar = calendar # UK calendar for ex-coupon calc
        ex_coupon_convention = ql.Unadjusted
        ex_coupon_eom = False

        comp = ql.Compounded
        freq = ql.Semiannual

        # Schedule for bond construction
        # C++ uses NullCalendar for schedule, then bond uses UK calendar for ex-coupon.
        # This might be slightly different from Python's FixedRateBond constructor if ex-coupon calendar is part of schedule.
        # However, exCouponPeriod is passed to FixedRateBond constructor.
        schedule_obj = ql.Schedule(start_date, maturity_date, tenor, ql.NullCalendar(),
                                   ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Forward,
                                   True, # EOM for schedule generation (as per C++ test)
                                   first_coupon_date) # Ensure first coupon date is matched

        dc = ql.ActualActual(ql.ActualActual.ISMA, schedule_obj)

        bond = ql.FixedRateBond(settlement_days, 100.0, schedule_obj, [coupon_rate], dc,
                                ql.Unadjusted, # Payment convention
                                100.0, # Redemption
                                issue_date,
                                # Ex-coupon parameters for FixedRateBond
                                ex_coupon_period,
                                ex_coupon_calendar, # Calendar for ex-coupon period calculation
                                ex_coupon_convention, # Convention for ex-coupon date adjustment
                                ex_coupon_eom) # End of month for ex-coupon

        leg = bond.cashflows()

        for case_data in cases:
            # Important: Set evaluation date for each case, as BondFunctions and CashFlows functions use it.
            ql.Settings.instance().evaluationDate = case_data.settlementDate

            accrued = bond.accruedAmount(case_data.settlementDate)
            assert_close(self, "accrued amount", case_data.settlementDate, accrued, case_data.accruedAmount, 1e-6)

            npv_from_test_price = case_data.testPrice + accrued
            assert_close(self, "NPV from test price", case_data.settlementDate, npv_from_test_price, case_data.NPV, 1e-6)

            # Note: CashFlows.yield takes includeSettlementDateFlows (default false)
            # and settlementDate (default evalDate).
            # The C++ test uses `false` and explicit settlementDate.
            # Python wrapper ql.CashFlows.yield_ uses bond.settlementDate() if settlementDate is None.
            # Here, the settlement date IS the evaluation date per loop.
            # We need to ensure the settlement date passed to yield is correct.
            # The C++ test uses `i.settlementDate` which is the current eval date.
            calc_yield = ql.CashFlows.yield_(leg, npv_from_test_price, dc, comp, freq, False, case_data.settlementDate, 1.0e-12, 100, 0.05)
            assert_close(self, "yield", case_data.settlementDate, calc_yield, case_data.yield_rate, 1e-6)

            duration = ql.CashFlows.duration(leg, ql.InterestRate(calc_yield, dc, comp, freq),
                                             ql.Duration.Modified, False, case_data.settlementDate)
            assert_close(self, "duration", case_data.settlementDate, duration, case_data.duration, 1e-6)

            convexity = ql.CashFlows.convexity(leg, ql.InterestRate(calc_yield, dc, comp, freq),
                                               False, case_data.settlementDate)
            assert_close(self, "convexity", case_data.settlementDate, convexity, case_data.convexity, 1e-6)

            # NPV from calculated yield
            calc_npv_from_yield = ql.CashFlows.npv(leg, ql.InterestRate(calc_yield, dc, comp, freq),
                                                   False, case_data.settlementDate)
            assert_close(self, "NPV from yield", case_data.settlementDate, calc_npv_from_yield, case_data.NPV, 1e-6)

            # Price from calculated NPV and accrued
            calc_price_from_yield = calc_npv_from_yield - accrued
            assert_close(self, "price from yield", case_data.settlementDate, calc_price_from_yield, case_data.testPrice, 1e-6)

        ql.Settings.instance().evaluationDate = original_eval_date


    def testExCouponAustralianBond(self):
        print("Testing ex-coupon Australian bond price against market values...")
        original_eval_date = ql.Settings.instance().evaluationDate

        class TestCaseData: # Re-using from Gilt test
            def __init__(self, settlementDate, testPrice, accruedAmount, NPV, yield_rate, duration, convexity):
                self.settlementDate = settlementDate; self.testPrice = testPrice
                self.accruedAmount = accruedAmount; self.NPV = NPV
                self.yield_rate = yield_rate; self.duration = duration; self.convexity = convexity

        cases = [
            TestCaseData(ql.Date(7,ql.August,2014), 103.0, 2.8670, 105.867, 0.04723, 2.26276, 6.54870),
            TestCaseData(ql.Date(8,ql.August,2014), 103.0, -0.1160, 102.884, 0.047235, 2.32536, 6.72531),
            TestCaseData(ql.Date(11,ql.August,2014), 103.0, -0.0660, 102.934, 0.04719, 2.31732, 6.68407)
        ]

        calendar = ql.Australia() # For settlement days if used, but not for ex-coupon period.
        settlement_days = 3

        issue_date = ql.Date(10, ql.June, 2004)
        start_date = ql.Date(15, ql.February, 2004) # Accrual start
        first_coupon_date = ql.Date(15, ql.August, 2004)
        maturity_date = ql.Date(15, ql.February, 2017)
        coupon_rate = 0.06
        tenor = ql.Period(6, ql.Months)
        # Ex-coupon: 7 calendar days, NullCalendar means strict calendar days.
        ex_coupon_period = ql.Period(7, ql.Days)
        ex_coupon_calendar = ql.NullCalendar() # Use NullCalendar for strict calendar day count
        ex_coupon_convention = ql.Unadjusted
        ex_coupon_eom = False

        comp = ql.Compounded
        freq = ql.Semiannual

        schedule_obj = ql.Schedule(start_date, maturity_date, tenor, ql.NullCalendar(),
                                   ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Forward,
                                   True, first_coupon_date)
        dc = ql.ActualActual(ql.ActualActual.ISMA, schedule_obj)

        bond = ql.FixedRateBond(settlement_days, 100.0, schedule_obj, [coupon_rate], dc,
                                ql.Unadjusted, 100.0, issue_date,
                                ex_coupon_period, ex_coupon_calendar,
                                ex_coupon_convention, ex_coupon_eom)

        leg = bond.cashflows()

        for case_data in cases:
            ql.Settings.instance().evaluationDate = case_data.settlementDate

            accrued = bond.accruedAmount(case_data.settlementDate)
            assert_close(self, "accrued amount", case_data.settlementDate, accrued, case_data.accruedAmount, 1e-3)

            npv_from_test_price = case_data.testPrice + accrued
            assert_close(self, "NPV from test price", case_data.settlementDate, npv_from_test_price, case_data.NPV, 1e-3)

            calc_yield = ql.CashFlows.yield_(leg, npv_from_test_price, dc, comp, freq, False, case_data.settlementDate, 1.0e-12, 100, 0.05)
            assert_close(self, "yield", case_data.settlementDate, calc_yield, case_data.yield_rate, 1e-5)

            duration = ql.CashFlows.duration(leg, ql.InterestRate(calc_yield, dc, comp, freq),
                                             ql.Duration.Modified, False, case_data.settlementDate)
            assert_close(self, "duration", case_data.settlementDate, duration, case_data.duration, 1e-5)

            convexity = ql.CashFlows.convexity(leg, ql.InterestRate(calc_yield, dc, comp, freq),
                                               False, case_data.settlementDate)
            assert_close(self, "convexity", case_data.settlementDate, convexity, case_data.convexity, 1e-4)

            calc_npv_from_yield = ql.CashFlows.npv(leg, ql.InterestRate(calc_yield, dc, comp, freq),
                                                   False, case_data.settlementDate)
            assert_close(self, "NPV from yield", case_data.settlementDate, calc_npv_from_yield, case_data.NPV, 1e-3)

            calc_price_from_yield = calc_npv_from_yield - accrued
            assert_close(self, "price from yield", case_data.settlementDate, calc_price_from_yield, case_data.testPrice, 1e-3)

        ql.Settings.instance().evaluationDate = original_eval_date

    def testBondFromScheduleWithDateVector(self):
        print("Testing South African R2048 bond price using Schedule constructor with Date vector...")
        original_eval_date = ql.Settings.instance().evaluationDate

        calendar = ql.NullCalendar() # For pricing from YTM
        settlement_days = 3 # For advancing eval date to settlement if needed, but settlementDate is explicit.

        issue_date = ql.Date(29, ql.June, 2012)
        today = ql.Date(7, ql.September, 2015)
        evaluation_date = calendar.adjust(today)
        settlement_date = calendar.advance(evaluation_date, settlement_days, ql.Days) # settlement_days * Days
        ql.Settings.instance().evaluationDate = evaluation_date

        maturity_date_raw = ql.Date(29, ql.February, 2048) # To generate Feb 29ths
        coupon_rate = 0.0875
        comp = ql.Compounded
        freq = ql.Semiannual
        tenor = ql.Period(6, ql.Months)
        ex_coupon_period = ql.Period(10, ql.Days)

        # Generate initial schedule
        schedule_raw = ql.Schedule(issue_date, maturity_date_raw, tenor, ql.NullCalendar(),
                                    ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, True)

        # Adjust Feb 29ths to Feb 28ths
        adjusted_dates = []
        for i in range(schedule_raw.size()):
            d = schedule_raw.date(i)
            if d.month() == ql.February and d.dayOfMonth() == 29:
                adjusted_dates.append(ql.Date(28, ql.February, d.year()))
            else:
                adjusted_dates.append(d)

        # Reconstruct schedule with adjusted dates
        # The C++ `Schedule(dates, calendar, convention, termConvention, tenor, rule, eom, isRegular)`
        # isRegular is missing in Python's `Schedule` constructor from list of dates.
        # We might need to check if the generated schedule is regular.
        # Python Schedule from list of dates: Schedule(std::vector< Date > const &dates, Calendar const &calendar=NullCalendar(), BusinessDayConvention convention=Unadjusted, BusinessDayConvention termDateConvention=Unadjusted, Period const &tenor=Period(), DateGeneration::Rule rule=DateGeneration::Backward, bool endOfMonth=false, std::vector< bool > const &isRegular=std::vector< bool >())
        # The `isRegular` part is available. The C++ test checks `schedule.isRegular()` - which would be a property of the original `schedule_raw`.
        # The new schedule from `adjusted_dates` might need `isRegular` determined or passed.
        # A safe assumption if not explicitly building `isRegular` vector is that QL handles it.
        # C++ passes `schedule.isRegular()` (a single bool) from the *old* schedule to the *new* one. This seems off.
        # It should be a vector. Let's assume the Python default is sufficient or determine regularity.
        # For now, let's try with defaults / minimal specification.

        # To mimic the C++ `schedule.isRegular()` logic for the last param, we need to check the *original* schedule.
        # `isRegular(i)` method on schedule object.
        # The final C++ Schedule constructor takes `std::vector<bool> isRegular = {}`
        # A single `bool` for `isRegular` in the C++ constructor `Schedule(dates, calendar, convention, termDateConvention, tenor, rule, eom, isRegular)`
        # usually means that *all* periods are regular or not. This seems like an overloaded constructor.
        # Let's use the one that takes vector<bool>.
        # The C++ code `schedule.isRegular()` returns a single bool, indicating if all periods have same length.
        # Let's assume default for `isRegular` vector for Python's `Schedule` from dates.

        schedule_final = ql.Schedule(adjusted_dates,
                                     schedule_raw.calendar(),
                                     schedule_raw.businessDayConvention(),
                                     schedule_raw.terminationDateBusinessDayConvention(),
                                     schedule_raw.tenor(), # Original tenor
                                     schedule_raw.rule(), # Original rule
                                     schedule_raw.endOfMonth()) # Original EOM
                                     # isRegular vector is omitted, using default behavior.

        dc = ql.ActualActual(ql.ActualActual.Bond, schedule_final)

        # FixedRateBond constructor in C++ for this case:
        # (settlementDays, faceAmount, schedule, coupons, accrualDayCounter, paymentConvention,
        #  redemption, issueDate, paymentCalendar, exCouponPeriod, exCouponCalendar,
        #  exCouponConvention, exCouponEndOfMonth)
        # Note: C++ test uses `0` for settlementDays here, implying it's handled by explicit settlement date in BondFunctions.
        bond = ql.FixedRateBond(0, # settlementDays for bond object construction
                               100.0, # faceAmount
                               schedule_final,
                               [coupon_rate],
                               dc,
                               ql.Following, # paymentConvention
                               100.0, # redemption
                               issue_date,
                               # Ex-coupon parameters:
                               ex_coupon_period,
                               calendar, # exCouponCalendar (NullCalendar)
                               ql.Unadjusted, # exCouponConvention
                               False) # exCouponEndOfMonth

        market_ytm_obj = ql.InterestRate(0.09185, dc, comp, freq)

        # BondFunctions::dirtyPrice(bond, yield, settlementDate)
        # The `yield` here is an InterestRate object.
        calculated_price = ql.BondFunctions.dirtyPrice(bond, market_ytm_obj, settlement_date)
        expected_price = 95.75706
        tolerance_price = 1e-5

        self.assertAlmostEqual(calculated_price, expected_price, delta=tolerance_price,
                               msg=(f"failed to reproduce R2048 dirty price"
                                    f"\n  expected:   {expected_price:.5f}"
                                    f"\n  calculated: {calculated_price:.5f}"))

        ql.Settings.instance().evaluationDate = original_eval_date


    def testFixedBondWithGivenDates(self):
        print("Testing fixed-coupon bond built on schedule with given dates...")
        vars_ = CommonVars()
        original_eval_date = ql.Settings.instance().evaluationDate

        today = ql.Date(22, ql.November, 2004)
        ql.Settings.instance().evaluationDate = today
        settlement_days = 1
        discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.03, ql.Actual360()))
        tolerance = 1.0e-6
        bond_engine = ql.DiscountingBondEngine(discount_curve)

        # Plain
        sch1_orig = ql.Schedule(ql.Date(30, ql.November, 2004), ql.Date(30, ql.November, 2008),
                                ql.Period(ql.Semiannual), ql.UnitedStates(ql.UnitedStates.GovernmentBond),
                                ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, False)
        bond1 = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch1_orig, [0.02875],
                                 ql.ActualActual(ql.ActualActual.ISMA), ql.ModifiedFollowing,
                                 100.0, ql.Date(30, ql.November, 2004))
        bond1.setPricingEngine(bond_engine)

        # Create schedule from dates of sch1_orig
        # The C++ `isRegular` vector seems to be all true here.
        is_regular_sch1 = [True] * (sch1_orig.size() -1) if sch1_orig.size() > 1 else []
        sch1_copy = ql.Schedule(sch1_orig.dates(), ql.UnitedStates(ql.UnitedStates.GovernmentBond),
                                ql.Unadjusted, ql.Unadjusted, ql.Period(ql.Semiannual),
                                ql.DateGeneration.Backward, False, is_regular_sch1)
        bond1_copy = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch1_copy, [0.02875],
                                      ql.ActualActual(ql.ActualActual.ISMA), ql.ModifiedFollowing,
                                      100.0, ql.Date(30, ql.November, 2004))
        bond1_copy.setPricingEngine(bond_engine)
        self.checkValue(bond1_copy.cleanPrice(), bond1.cleanPrice(), tolerance,
                        "Price mismatch for fixed bond 1 (plain, schedule from dates):")

        # Varying coupons
        coupon_rates = [0.02875, 0.03, 0.03125, 0.0325]
        bond2 = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch1_orig, coupon_rates,
                                 ql.ActualActual(ql.ActualActual.ISMA), ql.ModifiedFollowing,
                                 100.0, ql.Date(30, ql.November, 2004))
        bond2.setPricingEngine(bond_engine)
        bond2_copy = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch1_copy, coupon_rates,
                                      ql.ActualActual(ql.ActualActual.ISMA), ql.ModifiedFollowing,
                                      100.0, ql.Date(30, ql.November, 2004))
        bond2_copy.setPricingEngine(bond_engine)
        self.checkValue(bond2_copy.cleanPrice(), bond2.cleanPrice(), tolerance,
                        "Price mismatch for fixed bond 2 (varying coupons, schedule from dates):")

        # Stub date
        sch3_orig = ql.Schedule(ql.Date(30, ql.November, 2004), ql.Date(30, ql.March, 2009),
                                ql.Period(ql.Semiannual), ql.UnitedStates(ql.UnitedStates.GovernmentBond),
                                ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, False,
                                ql.Date(), ql.Date(30, ql.November, 2008)) # NextToLastDate, EndDate
        bond3 = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch3_orig, coupon_rates,
                                 ql.Actual360(), ql.ModifiedFollowing,
                                 100.0, ql.Date(30, ql.November, 2004))
        bond3.setPricingEngine(bond_engine)

        is_regular_sch3 = [True] * (sch3_orig.size() -1) if sch3_orig.size() > 1 else []
        sch3_copy = ql.Schedule(sch3_orig.dates(), ql.UnitedStates(ql.UnitedStates.GovernmentBond),
                                ql.Unadjusted, ql.Unadjusted, ql.Period(ql.Semiannual),
                                ql.DateGeneration.Backward, False, is_regular_sch3)
        bond3_copy = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch3_copy, coupon_rates,
                                      ql.Actual360(), ql.ModifiedFollowing,
                                      100.0, ql.Date(30, ql.November, 2004))
        bond3_copy.setPricingEngine(bond_engine)
        self.checkValue(bond3_copy.cleanPrice(), bond3.cleanPrice(), tolerance,
                        "Price mismatch for fixed bond 3 (stub date, schedule from dates):")

        ql.Settings.instance().evaluationDate = original_eval_date


    def testRiskyBondWithGivenDates(self):
        print("Testing risky bond engine...")
        vars_ = CommonVars()
        original_eval_date = ql.Settings.instance().evaluationDate

        today = ql.Date(22, ql.November, 2005)
        ql.Settings.instance().evaluationDate = today

        hazard_rate_quote = ql.QuoteHandle(ql.SimpleQuote(0.1))
        default_probability = ql.DefaultProbabilityTermStructureHandle(
            ql.FlatHazardRate(0, ql.TARGET(), hazard_rate_quote, ql.Actual360()))

        risk_free_handle = ql.RelinkableYieldTermStructureHandle()
        risk_free_handle.linkTo(ql.FlatForward(today, 0.02, ql.Actual360()))

        sch1 = ql.Schedule(ql.Date(30, ql.November, 2004), ql.Date(30, ql.November, 2008),
                           ql.Period(ql.Semiannual), ql.UnitedStates(ql.UnitedStates.GovernmentBond),
                           ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, False)

        settlement_days = 1
        # Notionals are not used in FixedRateBond constructor directly this way.
        # If variable notionals are needed, a more generic Bond object or custom leg is required.
        # The C++ test seems to declare `notionals` but doesn't use it for `FixedRateBond`.
        # It's used for the `couponRates` vector length, so it implies 4 coupon periods.
        # Wait, `couponRates` has 4 elements, but schedule from 2004 to 2008 semiannual is 8 periods.
        # FixedRateBond will cycle through `couponRates` if it's shorter than number of coupons.
        # The C++ test has `couponRates` with 4 elements.
        # Let's use what's in the C++ test for FixedRateBond construction.

        coupon_rates = [0.02875, 0.03, 0.03125, 0.0325] # Will be cycled for 8 coupons
        recovery_rate = 0.4

        bond = ql.FixedRateBond(settlement_days, vars_.faceAmount, sch1, coupon_rates,
                                ql.ActualActual(ql.ActualActual.ISMA), ql.ModifiedFollowing, 100.0,
                                ql.Date(20, ql.November, 2004)) # Issue date

        bond_engine = ql.RiskyBondEngine(default_probability, recovery_rate, risk_free_handle)
        bond.setPricingEngine(bond_engine)

        expected_npv = 888458.819055
        calculated_npv = bond.NPV()
        tolerance = 1.0e-6
        self.checkValue(calculated_npv, expected_npv, tolerance, "Failed to reproduce risky bond NPV:")

        expected_price = 87.407883 # This seems to be per 100 face
        calculated_price = bond.cleanPrice()
        self.checkValue(calculated_price, expected_price, tolerance, "Failed to reproduce risky bond price:")

        ql.Settings.instance().evaluationDate = original_eval_date

    def testFixedRateBondWithArbitrarySchedule(self):
        print("Testing fixed-rate bond with arbitrary schedule...")
        original_eval_date = ql.Settings.instance().evaluationDate

        calendar = ql.NullCalendar()
        settlement_days = 3
        today = ql.Date(1, ql.January, 2019)
        ql.Settings.instance().evaluationDate = today

        dates = [
            ql.Date(1, ql.February, 2019), ql.Date(7, ql.February, 2019),
            ql.Date(1, ql.April, 2019), ql.Date(27, ql.May, 2019)
        ]
        schedule = ql.Schedule(dates, calendar, ql.Unadjusted) # No other params implies default for rest

        coupon_rate = 0.01
        dc = ql.Actual365Fixed()

        bond = ql.FixedRateBond(settlement_days, 100.0, schedule, [coupon_rate], dc,
                                ql.Following, 100.0) # Default issue date if not specified.

        self.assertEqual(bond.frequency(), ql.NoFrequency, f"unexpected frequency: {bond.frequency()}")

        discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.03, ql.Actual360()))
        bond.setPricingEngine(ql.DiscountingBondEngine(discount_curve))

        try:
            bond.cleanPrice() # BOOST_CHECK_NO_THROW equivalent
        except Exception as e:
            self.fail(f"bond.cleanPrice() raised an exception: {e}")

        ql.Settings.instance().evaluationDate = original_eval_date

    def testThirty360BondWithSettlementOn31st(self):
        print("Testing Thirty/360 bond with settlement on 31st of the month...")
        original_eval_date = ql.Settings.instance().evaluationDate
        ql.Settings.instance().evaluationDate = ql.Date(28, ql.July, 2017)

        dated_date = ql.Date(13, ql.February, 2014)
        settlement_date = ql.Date(31, ql.July, 2017)
        maturity_date = ql.Date(13, ql.August, 2018)

        day_counter = ql.Thirty360(ql.Thirty360.USA)
        compounding = ql.Compounded
        frequency = ql.Semiannual # For yield calculation

        fixed_bond_schedule = ql.Schedule(
            dated_date, maturity_date, ql.Period(frequency),
            ql.UnitedStates(ql.UnitedStates.GovernmentBond),
            ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Forward, False)

        fixed_rate_bond = ql.FixedRateBond(
            1, 100.0, fixed_bond_schedule, [0.015],
            day_counter, ql.Unadjusted, 100.0)

        clean_price_obj = ql.BondPrice(100.0, ql.BondPrice.Clean)

        calc_yield = ql.BondFunctions.yield_(
            fixed_rate_bond, clean_price_obj.amount(), day_counter, compounding, frequency,
            settlement_date, 1e-12, 100, 0.05, clean_price_obj.type()) # Price type
        assert_close(self, "yield", settlement_date, calc_yield, 0.015, 1e-4)

        interest_rate_obj = ql.InterestRate(calc_yield, day_counter, compounding, frequency)
        duration = ql.BondFunctions.duration(fixed_rate_bond, interest_rate_obj, ql.Duration.Macaulay, settlement_date)
        assert_close(self, "duration", settlement_date, duration, 1.022, 1e-3)

        convexity = ql.BondFunctions.convexity(fixed_rate_bond, interest_rate_obj, settlement_date) / 100.0
        assert_close(self, "convexity", settlement_date, convexity, 0.015, 1e-3)

        accrued = ql.BondFunctions.accruedAmount(fixed_rate_bond, settlement_date)
        assert_close(self, "accrued", settlement_date, accrued, 0.7, 1e-6)

        ql.Settings.instance().evaluationDate = original_eval_date

    def testBasisPointValue(self):
        print("Testing consistency of bond basisPointValue and yieldValueBasisPoint calculations...")
        vars_ = CommonVars() # Uses its own today, sets eval date
        original_eval_date = ql.Settings.instance().evaluationDate

        today = ql.Date(29, ql.January, 2024)
        ql.Settings.instance().evaluationDate = today

        dated_date = ql.Date(15, ql.November, 2023)
        maturity_date = ql.Date(15, ql.August, 2033)

        day_counter = ql.Thirty360(ql.Thirty360.USA)
        compounding = ql.Compounded
        frequency = ql.Semiannual
        period = ql.Period(frequency)

        fixed_bond_schedule = ql.Schedule(
            dated_date, maturity_date, period,
            ql.UnitedStates(ql.UnitedStates.GovernmentBond),
            ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Forward, False)

        fixed_rate_bond = ql.FixedRateBond(
            1, vars_.faceAmount, fixed_bond_schedule, [0.045],
            day_counter, ql.Unadjusted, 100.0)

        default_settlement = fixed_rate_bond.settlementDate()
        clean_price_obj = ql.BondPrice(102.890625, ql.BondPrice.Clean)
        tolerance = 1e-6

        # Calculate yield based on the default settlement date
        # The C++ test calls `yield` without explicit settlement, so it uses default bond settlement.
        calc_yield = ql.BondFunctions.yield_(
            fixed_rate_bond, clean_price_obj.amount(), day_counter, compounding, frequency,
            ql.Date(), 1e-12, 100, 0.05, clean_price_obj.type()) # Default settlement used

        assert_close(self, "yield", default_settlement, calc_yield, 0.041301, tolerance)

        class TestCaseBPV:
            def __init__(self, settlement, bpv, yvbp):
                self.settlement = settlement; self.bpv = bpv; self.yvbp = yvbp

        cases = [
            TestCaseBPV(ql.Date(), -795.459834, -0.0012571287), # Default settlement date
            TestCaseBPV(default_settlement, -795.459834, -0.0012571287),
            TestCaseBPV(ql.Date(12, ql.February, 2024), -793.149033, -0.0012607913),
        ]

        interest_rate_obj = ql.InterestRate(calc_yield, day_counter, compounding, frequency)

        for case in cases:
            # Use the `calc_yield` (float) for functions taking yield as Rate
            # Use `interest_rate_obj` for functions taking InterestRate object

            # basisPointValue from yield (Rate)
            bpv1 = ql.BondFunctions.basisPointValue(fixed_rate_bond, calc_yield, day_counter, compounding, frequency, case.settlement)
            assert_close(self, "basisPointValue from yield", case.settlement, bpv1, case.bpv, tolerance)

            # basisPointValue from InterestRate object
            bpv2 = ql.BondFunctions.basisPointValue(fixed_rate_bond, interest_rate_obj, case.settlement)
            assert_close(self, "basisPointValue from InterestRate", case.settlement, bpv2, case.bpv, tolerance)

            # yieldValueBasisPoint from yield (Rate)
            # Note: C++ test multiplies by faceAmount *after* call. Python wrapper might do it internally or expect it.
            # Let's assume it returns per 100 face, then scale, as in C++.
            yvbp1_per100 = ql.BondFunctions.yieldValueBasisPoint(fixed_rate_bond, calc_yield, day_counter, compounding, frequency, case.settlement)
            yvbp1 = yvbp1_per100 * (vars_.faceAmount / 100.0) # Scale to full face amount
            assert_close(self, "yieldValueBasisPoint from yield", case.settlement, yvbp1, case.yvbp * vars_.faceAmount, tolerance) # Expected yvbp is also scaled

            # yieldValueBasisPoint from InterestRate object
            yvbp2_per100 = ql.BondFunctions.yieldValueBasisPoint(fixed_rate_bond, interest_rate_obj, case.settlement)
            yvbp2 = yvbp2_per100 * (vars_.faceAmount / 100.0)
            assert_close(self, "yieldValueBasisPoint from InterestRate", case.settlement, yvbp2, case.yvbp * vars_.faceAmount, tolerance)

        ql.Settings.instance().evaluationDate = original_eval_date


if __name__ == '__main__':
    print("Presolve testQuantLib.py ...")
    unittest.main(argv=['first-arg-is-ignored'], exit=False)